In [1]:
## ---------------------- START if you want to use google colab, ---------------------- ##
# !pip install pytorch_transformers
# !pip install pytorch_pretrained_bert

     |████████████████████████████████| 184kB 8.9MB/s 
     |████████████████████████████████| 1.0MB 54.2MB/s 
     |████████████████████████████████| 870kB 52.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b00cbebd0f5c63e94ed7fb22e5af15a6d113fb29e1b394ebfbb4538d45b57a87
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 133kB 9.2MB/s 


In [2]:
# from google.colab import drive
# drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
# cd drive/My\ Drive/'YOUR DATA_PATH'

/content/drive/My Drive/Machine_Learning/CSE576/PJ/clicr/dataset-code


In [4]:
# !pwd

/content/drive/My Drive/Machine_Learning/CSE576/PJ/clicr/dataset-code


In [ ]:
## ---------------------- END if you want to use google colab, ---------------------- ##

In [1]:
import os
import json
import pandas as pd
import numpy as np
from pytorch_transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange


In [2]:
!nvidia-smi

Mon Apr 13 15:43:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   31C    P0    56W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from create_processed_data import *

Using TensorFlow backend.


In [4]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [6]:
device

device(type='cuda')

In [7]:
torch.cuda.get_device_name(0) 

'Tesla V100-SXM2-16GB'

In [8]:
DATASET_SIZE = 53914
MAX_LEN = 424
SEED = 520
bs = 16

In [10]:
train_data_processor = DataProcessor('train',size=32,tokenizer=None)
tags_vals = train_data_processor.get_tags_vals()

In [11]:
tr_inputs,tr_masks,tr_tags,tr_ttids,train_tokenizer = train_data_processor.get_processed_data()

In [12]:
valid_data_processor = DataProcessor('dev',size=32,tokenizer=train_tokenizer)

In [13]:
val_inputs,val_masks,val_tags,val_ttids,train_val_tokenizer = valid_data_processor.get_processed_data()

In [14]:
tr_inputs = torch.tensor(tr_inputs)
tr_tags = torch.tensor(tr_tags)
tr_masks = torch.tensor(tr_masks)
tr_ttids = torch.tensor(tr_ttids)
val_inputs = torch.tensor(val_inputs)
val_tags = torch.tensor(val_tags)
val_masks = torch.tensor(val_masks)
val_ttids = torch.tensor(val_ttids)

In [15]:
torch.cuda.empty_cache() 

In [16]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags, tr_ttids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)
valid_data = TensorDataset(val_inputs, val_masks, val_tags, val_ttids)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [17]:
!nvidia-smi

Mon Apr 13 15:43:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   30C    P0    41W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [18]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tags_vals))



#output_dir = "./models/"
# Step 2: Re-load the saved model and vocabulary

# Example for a Bert model
#model = BertForTokenClassification.from_pretrained(output_dir,num_labels=len(tag2idx))

In [19]:
model.cuda();

In [20]:
!nvidia-smi

Mon Apr 13 15:43:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   31C    P0    56W / 300W |   1472MiB / 16160MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [21]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=1e-4)

In [22]:
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report
from torch.nn import CrossEntropyLoss

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [43]:
epochs = 12
max_grad_norm = 1.0
F_scores = np.zeros(epochs,float)
patience = 6
for e in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels, b_ttids = batch
        # forward pass
        #loss = model(b_input_ids, token_type_ids=None,attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask)
        closs = CrossEntropyLoss(ignore_index=-100)
        logits = logits.view(-1,len(tags_vals))
        b_labels = b_labels.view(-1)
        loss = closs(logits, b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels, b_ttids = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=b_ttids,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=b_ttids,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i if l_ii!=-100]
    c_f1 = f1_score(valid_tags, pred_tags)
    rep = classification_report(valid_tags, pred_tags)
    print(rep)
    print("F1-Score: {}".format(c_f1))
    #k epochs no improvement
    F_scores[e] = c_f1
    prev_Fs = np.arange(e-patience,e)
    prev_indices = prev_Fs[prev_Fs>=0]
    if all(F_scores[e] - F_scores[prev_indices] < 0 ) and e > patience:
        print('Done training')
        break
    else:
        print('Still training')
        #save checkpoint


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.4930349811911583


Epoch:  10%|█         | 1/10 [00:02<00:19,  2.21s/it]

Validation loss: 0.07491471618413925
Validation Accuracy: 0.5916863207547169
           precision    recall  f1-score   support

      ans       0.00      0.00      0.00        42

micro avg       0.00      0.00      0.00        42
macro avg       0.00      0.00      0.00        42

F1-Score: 0
Still training
Train loss: 0.053221301486094795


Epoch:  20%|██        | 2/10 [00:03<00:16,  2.08s/it]

...

Validation loss: 0.07629457488656044
Validation Accuracy: 0.5916863207547169
           precision    recall  f1-score   support

      ans       0.00      0.00      0.00        42

micro avg       0.00      0.00      0.00        42
macro avg       0.00      0.00      0.00        42

F1-Score: 0
Still training
Train loss: 0.0470652903119723


Epoch:  90%|█████████ | 9/10 [00:16<00:01,  1.81s/it]

Validation loss: 0.07769349962472916
Validation Accuracy: 0.5916863207547169
           precision    recall  f1-score   support

      ans       0.00      0.00      0.00        42

micro avg       0.00      0.00      0.00        42
macro avg       0.00      0.00      0.00        42

F1-Score: 0
Still training
Train loss: 0.049621492313841976


Epoch: 100%|██████████| 10/10 [00:18<00:00,  1.81s/it]

Validation loss: 0.07957742363214493
Validation Accuracy: 0.5916863207547169
           precision    recall  f1-score   support

      ans       0.00      0.00      0.00        42

micro avg       0.00      0.00      0.00        42
macro avg       0.00      0.00      0.00        42

F1-Score: 0
Still training


In [47]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels, b_ttids = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=b_ttids,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=b_ttids,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i if l_ii!=-100] for l in true_labels for l_i in l]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
rep = classification_report(valid_tags, pred_tags)
print(rep)

Validation loss: 0.07957742363214493
Validation Accuracy: 0.5916863207547169
Validation F1-Score: 0
           precision    recall  f1-score   support

      ans       0.00      0.00      0.00        42

micro avg       0.00      0.00      0.00        42
macro avg       0.00      0.00      0.00        42



In [129]:
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./models2/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

'./models2/vocab.txt'

2

9